In [1]:
import os, glob
import pandas as pd
from io import StringIO
import numpy as np
import pysam
import pybedtools 
from Bio import SeqIO
from Bio.Seq import Seq
pybedtools.helpers.set_tempdir("/data/projects/temp")

In [2]:
non_coding_region = "donor"

In [3]:
data_path = '/data/projects/DNABERT_snv/Manuscript_11_2023/Intersected_data/{}_ICGC_intersected.tsv'.format(non_coding_region)
output_path = "/data/projects/DNABERT_snv/Manuscript_11_2023/DNABERT_data/{}".format(non_coding_region) 

In [4]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [5]:
def seq2kmer(seq):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    k=6
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [6]:
genome = SeqIO.to_dict(SeqIO.parse("/data/projects/Resources/Gencode_genome_annotation/GRCh38.primary_assembly.genome.fa", "fasta"))

In [28]:
df = pd.read_csv(data_path, sep= '\t')
print(df.shape)

(5792692, 22)


In [29]:
# df = pd.read_csv(data_path, sep= '\t')
# print(df.shape)

# df = df[df['No_of_variations']<3].reset_index(drop=True)
# print(df.shape)
# df = df.loc[df['Alternative_Allele']== '.'].reset_index(drop=True)
# df
#df= df[df['strand']=="-"]


data= []
#df = df.head(10)
# Iterate over DataFrame rows
for idx, row in df.iterrows():
    try:
        #print(row)Reference_Allele	Alternative_Allele
        alts = row['Alternative_Allele'].rstrip(',').split(',')
        ref_seq = str(genome[row['chr_name']].seq[ row['Donor_start']:row['Donor_end']])

        # Calculate the variant position relative to the fetched sequence
        variant_pos_start = row['Start'] - row['Donor_start']
        variant_pos_end = row['End']  - row['Donor_start']
        # print(variant_pos_start, variant_pos_end, alts,  row['Reference_Allele'])
        # print(ref_seq)
        # if row['strand'] == '-':
        #     ref_seq = str(Seq(ref_seq).reverse_complement())
        #print(ref_seq)


        #Replace the reference nucleotide with the alternate nucleotide to get the alternate sequence
        for alt in alts:
            if alt == ".":  # Deletion
                # Fetch additional bases from genome to maintain sequence length
                extra_bases = str(genome[row['chr_name']].seq[row['Donor_end']:row['Donor_end'] + len(row['Reference_Allele'])])
                #print(len(row['Reference_Nucleotide']))
                alt_seq = ref_seq[:variant_pos_start] + ref_seq[variant_pos_end:] + extra_bases
            else:  # SNPs, Insertions, and partial Deletions
                extra_bases = str(genome[row['chr_name']].seq[row['Donor_end']:row['Donor_end'] + len(row['Reference_Allele']) - len(alt)])
                #print(extra_bases)
                if(variant_pos_start>=0):
                    alt_seq = ref_seq[:variant_pos_start] + alt + ref_seq[variant_pos_end:] + extra_bases
                else:
                    continue
            #print(alt_seq)
            #Append to the list as a dictionary
            data.append({
                'chr': row['chr_name'],
                'strand': row['strand'],
                'Donor_coordinates': str(row['Donor_start'])+'-'+str(row['Donor_end']),
                'Ensemble_Transcript_ID': row['ENSEMBL_Transcript_ID'],
                'Mutation_ID': row['Mutation_ID'],
                'varinat_start':  row['Start'],
                'variant_end':  row['End'],
                'ref_neucleotide': row['Reference_Allele'],
                'alternative_neucleotide': alt,
                'reference_seq': ref_seq,
                'alt_seq': alt_seq
            })
            
    except KeyError:
        print("NA")
        # # If sequence is not present, append a default value
        # ref_sequences.append("NA")
        # alt_sequences.append("NA")

        
        
new_df = pd.DataFrame(data)
print(new_df.shape)
new_df = new_df.drop_duplicates().reset_index(drop=True)
print(new_df.shape)
merged_list = list(zip(new_df['reference_seq'], new_df['alt_seq']))
merged_list = [item.upper() for tup in merged_list for item in tup]
#print(merged_list)
kmer_lst = list(map(seq2kmer, merged_list))
df_kmer = pd.DataFrame(kmer_lst, columns=['Sequence'])
df_kmer['Label'] = np.random.choice([0, 1], size=len(df_kmer))

new_df.to_csv(output_path + "/all_data_new.tsv", sep="\t", index= False)
df_kmer.to_csv(output_path + "/dev.tsv", sep="\t", index= False)

(5786746, 11)
(5783396, 11)


In [30]:
new_df

,chr,strand,Donor_coordinates,Ensemble_Transcript_ID,Mutation_ID,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,reference_seq,alt_seq
0,chr7,+,127588526-127588605,ENST00000000233,MU80692579,127588529,127588530,G,T,CGCGGATCTTCGGGAAGAAGCAGATGCGGATTCTCATGGGTGAGGC...,CGCTGATCTTCGGGAAGAAGCAGATGCGGATTCTCATGGGTGAGGC...
1,chr7,+,127588526-127588605,ENST00000000233,MU90753368,127588553,127588554,G,A,CGCGGATCTTCGGGAAGAAGCAGATGCGGATTCTCATGGGTGAGGC...,CGCGGATCTTCGGGAAGAAGCAGATGCAGATTCTCATGGGTGAGGC...
2,chr7,+,127588526-127588605,ENST00000000233,MU115107606,127588569,127588570,G,T,CGCGGATCTTCGGGAAGAAGCAGATGCGGATTCTCATGGGTGAGGC...,CGCGGATCTTCGGGAAGAAGCAGATGCGGATTCTCATGGGTGATGC...
3,chr7,+,127588526-127588605,ENST00000000233,MU84868698,127588581,127588582,G,T,CGCGGATCTTCGGGAAGAAGCAGATGCGGATTCTCATGGGTGAGGC...,CGCGGATCTTCGGGAAGAAGCAGATGCGGATTCTCATGGGTGAGGC...
4,chr7,+,127588526-127588605,ENST00000000233,MU83704948,127588591,127588592,G,T,CGCGGATCTTCGGGAAGAAGCAGATGCGGATTCTCATGGGTGAGGC...,CGCGGATCTTCGGGAAGAAGCAGATGCGGATTCTCATGGGTGAGGC...
...,...,...,...,...,...,...,...,...,...,...,...
5783391,chr17,-,63710688-63710767,ENST00000640999,MU28664773,63710752,63710753,C,A,CCAATAACCCCTTTCTACCAAGAACCCTTTCCCACTCACCGTATGC...,CCAATAACCCCTTTCTACCAAGAACCCTTTCCCACTCACCGTATGC...
5783392,chr17,-,63710688-63710767,ENST00000640999,MU29299841,63710760,63710761,C,T,CCAATAACCCCTTTCTACCAAGAACCCTTTCCCACTCACCGTATGC...,CCAATAACCCCTTTCTACCAAGAACCCTTTCCCACTCACCGTATGC...
5783393,chr17,-,63710688-63710767,ENST00000640999,MU129502912,63710765,63710766,T,C,CCAATAACCCCTTTCTACCAAGAACCCTTTCCCACTCACCGTATGC...,CCAATAACCCCTTTCTACCAAGAACCCTTTCCCACTCACCGTATGC...
5783394,chr17,-,63713366-63713445,ENST00000640999,MU84546198,63713396,63713397,G,T,CCCACCCTCCCTCCATGTGACACACTCATGGTTTATTACCTGCAAG...,CCCACCCTCCCTCCATGTGACACACTCATGTTTTATTACCTGCAAG...


In [31]:
df_kmer

,Sequence,Label
0,CGCGGA GCGGAT CGGATC GGATCT GATCTT ATCTTC TCTT...,1
1,CGCTGA GCTGAT CTGATC TGATCT GATCTT ATCTTC TCTT...,0
2,CGCGGA GCGGAT CGGATC GGATCT GATCTT ATCTTC TCTT...,1
3,CGCGGA GCGGAT CGGATC GGATCT GATCTT ATCTTC TCTT...,1
4,CGCGGA GCGGAT CGGATC GGATCT GATCTT ATCTTC TCTT...,1
...,...,...
11566787,CCAATA CAATAA AATAAC ATAACC TAACCC AACCCC ACCC...,0
11566788,CCCACC CCACCC CACCCT ACCCTC CCCTCC CCTCCC CTCC...,1
11566789,CCCACC CCACCC CACCCT ACCCTC CCCTCC CCTCCC CTCC...,1
11566790,CCCACC CCACCC CACCCT ACCCTC CCCTCC CCTCCC CTCC...,1
